## Table Of Contents
<font size=3rem>
    
0 -**[ GPU SETTINGS](#GPU-SETTINGS)**<br>
1 -**[ BUSINESS UNDERSTANDING](#BUSINESS-UNDERSTANDING)**<br>
2 -**[ DATA UNDERSTANDING](#DATA-UNDERSTANDING)**<br>
3 -**[ DATA PREPARATION](#DATA-PREPARATION)**<br>
4 -**[ MODELING](#MODELING)**<br>
5 -**[ EVALUATION](#EVALUATION)<br>**
</font>
___

# BUSINESS UNDERSTANDING

- Student: Cody Freese
- Pace: Self Paced
- Jeff Herman

- Determine the difference between optical illusions and reality based images.

## Import Tools

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten
import cv2
import random
import datetime
from tensorflow.python.client import device_lib
from tensorflow.keras.applications.densenet import DenseNet201

Using TensorFlow backend.


In [2]:
notebook_original_start = datetime.datetime.now()

# GPU SETTINGS

In [3]:
# https://github.com/jeffheaton/present/blob/master/youtube/gpu/keras-dual-gpu.ipynb
# https://www.youtube.com/watch?v=HCLmM1PyDIs

# List of Local Devices
devices = device_lib.list_local_devices()

# Calculate size of device
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

# Data in Devices, Type, Name, Abrv, Memory
for d in devices:
    t = d.device_type
    name = d.physical_device_desc
    l = [item.split(':',1) for item in name.split(", ")]
    name_attr = dict([x for x in l if len(x)==2])
    dev = name_attr.get('name', 'Unnamed device')
    print(f" {d.name} || {dev} || {t} || {sizeof_fmt(d.memory_limit)}")

 /device:CPU:0 || Unnamed device || CPU || 256.0 MiB
 /device:GPU:0 ||  NVIDIA GeForce RTX 2070 SUPER || GPU || 5.8 GiB


In [4]:
print("Number of GPU's Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Number of GPU's Available:  1


## FUNCTIONS

In [5]:
def get_num_pixels(pixel_path):
    '''
    Gets width and height of image
    :param pixel_path: String
    :return width, height: Tuple of Int
    '''
    
    width, height = Image.open(pixel_path).size
    return width, height

In [6]:
def get_image_pixels_and_paths(path):
    '''
    Joins the classes of called picture pixel count for total of called class
    :param path: String
    :return None: prints out pixel dimensions
    '''
    
    # Iterate through 5 entries to print number of pixels from image
    for path, subdirs, files in os.walk(path):
        for index, name in enumerate(files):
            image_path = os.path.join(path,name)
            print(get_num_pixels(image_path))
            if index >= 4:
                break

In [ ]:
# Code Sourced from Learn.Co, modified for this project
# https://github.com/learn-co-curriculum/dsc-image-classification-with-mlps-lab/tree/solution
def visualize_training_results(results):
    '''
    Renders out metric results into graphs
    :param results: tensorflow.python.keras.callbacks.History
    :return None: print out graphs
    '''
    # Assign variable for model history results
    history = results.history
    
    # Loss metric
    plt.figure()
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.legend(['loss','val_loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    # Accuracy metric
    plt.figure()
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.legend(['accuracy','val_accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()
    
    # Precision metric
    plt.figure()
    plt.plot(history['precision'])
    plt.plot(history['val_precision'])
    plt.legend(['precision','val_precision'])
    plt.title('Precision')
    plt.xlabel('Epochs')
    plt.ylabel('Precision')
    plt.show()
    
    # Recall metric
    plt.figure()
    plt.plot(history['recall'])
    plt.plot(history['val_recall'])
    plt.legend(['recall','val_recall'])
    plt.title('Recall')
    plt.xlabel('Epochs')
    plt.ylabel('Recall')
    plt.show()

 # DATA UNDERSTANDING

In [ ]:
#Home Directory
home_path = r'C:\Users\c_fre\Learn.co\Module_5_Project\Illusion-Image-Classification\illusion_images'

#Train Directory
train_path = os.path.join(home_path,'train')

#Validation Directory
val_path = os.path.join(home_path,'val')

#Test Directory
test_path = os.path.join(home_path,'test')

#Train Subdirectories
normal_train_path = os.path.join(train_path,'NORMAL')

pneumonia_train_path = os.path.join(train_path,'ILLUSION')

#Validation Subdirectories
normal_val_path = os.path.join(val_path,'NORMAL')

pneumonia_val_path = os.path.join(val_path,'ILLUSION')

#Test Subdirectories

normal_test_path = os.path.join(test_path,'NORMAL')

pneumonia_test_path = os.path.join(test_path,'ILLUSION')

In [ ]:
#Print to Verify Directory Pathways
print('Directory Contents:',os.listdir(home_path))
print('train Contents:',os.listdir(train_path))
print('val Contents:',os.listdir(val_path))
print('test Contents:',os.listdir(test_path))

In [ ]:
#Cases in Training Sets
print("Number of cases in Training:",len(os.listdir(normal_train_path)) +len(os.listdir(pneumonia_train_path)))

#Pneumonia Cases in Training
print("Number of Pneumonia cases in Training:",len(os.listdir(pneumonia_train_path)))

#Normal Cases in Training
print("Number of Normal cases in Training:",len(os.listdir(normal_train_path)))

In [ ]:
# Cases in Validation Sets
print("Number of cases in Validation:",len(os.listdir(normal_val_path)) +len(os.listdir(pneumonia_val_path)))

# Pneumonia Cases in Validation
print("Number of Pneumonia cases in Validation:",len(os.listdir(pneumonia_val_path)))

# Normal Cases in Validation
print("Number of Normal cases in Validation:",len(os.listdir(normal_val_path)))

In [ ]:
#Cases in Test Sets
print("Number of cases in Test:",len(os.listdir(normal_test_path)) +len(os.listdir(pneumonia_test_path)))

#Pneumonia Cases in Test
print("Number of Pneumonia cases in Test:",len(os.listdir(pneumonia_test_path)))

#Normal cases in Test
print("Number of Normal cases in Test:",len(os.listdir(normal_test_path)))

In [ ]:
# #Total number of Normal Cases
print("Total number of Normal cases:",
      len(os.listdir(normal_test_path))+
      len(os.listdir(normal_train_path))+
      len(os.listdir(normal_val_path)))

#Total number of Pneumonia Cases
print("Total number of Pneumonia cases:",
      len(os.listdir(pneumonia_test_path))+
      len(os.listdir(pneumonia_train_path))+
      len(os.listdir(pneumonia_val_path)))

In [ ]:
# Validation Pixel Size
get_image_pixels_and_paths(val_path)

In [ ]:
# Display all images in Normal Validatoin pathway
normal_val_images_display = os.listdir(normal_val_path)[:5]
for image_name in normal_val_images_display:
    image_path = os.path.join(normal_val_path, image_name)
    img = image.load_img(image_path)
    plt.imshow(img)
    plt.show()

In [ ]:
# Display all images in Pneumonia validation pathway
pneumonia_val_image_display = os.listdir(pneumonia_val_path)[:5]
for image_name in pneumonia_val_image_display:
    image_path = os.path.join(pneumonia_val_path, image_name)
    img = image.load_img(image_path)
    plt.imshow(img)
    plt.show()

# DATA PREPARATION

In [ ]:
# Double default batch size due to computer hardware
batch_size = 64

# Set all image sizes universally
img_width, img_height = 200,200

In [ ]:
# Augmentation of Training data, rescale
train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest")


# Rescale validation data
val_datagen = ImageDataGenerator(rescale=1 / 255.0)

# Rescale test data
test_datagen = ImageDataGenerator(rescale=1 / 255.0)

In [ ]:
# Training Generator
train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
    seed=7539541
)

# Validation Generator
valid_generator = val_datagen.flow_from_directory(
    directory=val_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
    seed=7539541
)

# Test Generator
test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=64,
    class_mode="binary",
    shuffle=True,
    seed=7539541
)

# MODELING

## Multi Layer Perceptron

In [ ]:
# Start time of model
original_start = datetime.datetime.now()
start = datetime.datetime.now()

In [ ]:
# Sequential: as we want to build layers upon each other
mlp_model = keras.Sequential()

In [ ]:
# Fuse paths without losing scalability
mlp_model.add(keras.layers.Flatten())

In [ ]:
# Dense: Fully Connected Layer, Reduce Gradient Vanish, Stop Inactive Neurons with relu
mlp_model.add(keras.layers.Dense(64,
                                 activation='relu'))

In [ ]:
# Dense: Fully Connected Layer, Reduce Gradient Vanish, Stop Inactive Neurons with relu
mlp_model.add(keras.layers.Dense(32,
                                 activation='relu'))

In [ ]:
# Dense: Fully Connected Layer, Reduce Gradient Vanish, Stop Inactive Neurons with relu
mlp_model.add(keras.layers.Dense(16,
                                 activation='relu'))

In [ ]:
# Dense: Fully Connected Layer, Reduce Gradient Vanish, Stop Inactive Neurons with relu
mlp_model.add(keras.layers.Dense(8,
                                 activation='relu'))

In [ ]:
# Full input, Binary output with sigmoid
mlp_model.add(keras.layers.Dense(1,
                                 activation='sigmoid'))

In [ ]:
# Compile. Binary Crossentropy to Predict Probability to actual class output, Stochastic Gradient Descent via Adam
mlp_model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy',
                           tf.keras.metrics.Precision(name='precision'),
                           tf.keras.metrics.Recall(name='recall')])

In [ ]:
# Fit Model, 10 Epochs
mlp_history = mlp_model.fit(train_generator,
                            validation_data = valid_generator,
                            steps_per_epoch = train_generator.n//train_generator.batch_size,
                            validation_steps = valid_generator.n//valid_generator.batch_size,
                            epochs=10)

# Stop time of model
end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
# Save Model Locally
mlp_model.save("MLP_Initial_Model")

In [ ]:
# Summary of Model
mlp_model.summary()

In [ ]:
# Visualize Model Metrics
visualize_training_results(mlp_history)

In [ ]:
# Evaluate against Training set
mlp_results_train = mlp_model.evaluate(train_generator)

In [ ]:
# Evaluate against Test set
mlp_results_test = mlp_model.evaluate(test_generator)

## CNN Model

In [ ]:
# Start time of model
original_start = datetime.datetime.now()
start = datetime.datetime.now()

In [ ]:
# Sequential: as we want to build layers upon each other
cnn_model = keras.Sequential()

In [ ]:
# 2D Convolution Layer over Images, Generate 64 filters and multiply across the image
# Filter Images to highlight features
cnn_model.add(keras.layers.Conv2D(64,
                                  kernel_size=(3,3),
                                  activation='relu',
                                  input_shape=(img_width, img_height, 3)))

In [ ]:
# Group images into 2x2, votes on largest, features maintained
cnn_model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# Fuse paths without losing scalability
cnn_model.add(keras.layers.Flatten())

In [ ]:
# Dense: Fully Connected Layer, Reduce Gradient Vanish, Stop Inactive Neurons with relu
cnn_model.add(keras.layers.Dense(64,
                                 activation='relu'))

In [ ]:
# Dense: Fully Connected Layer, Reduce Gradient Vanish, Stop Inactive Neurons with relu
cnn_model.add(keras.layers.Dense(32,
                                 activation='relu'))

In [ ]:
# Dense: Fully Connected Layer, Reduce Gradient Vanish, Stop Inactive Neurons with relu
cnn_model.add(keras.layers.Dense(16,
                                 activation='relu'))

In [ ]:
# Dense: Fully Connected Layer, Reduce Gradient Vanish, Stop Inactive Neurons with relu
cnn_model.add(keras.layers.Dense(8,
                                 activation='relu'))

In [ ]:
# Full input, Binary output with sigmoid
cnn_model.add(keras.layers.Dense(1,
                                 activation='sigmoid'))

In [ ]:
# Compile. Binary Crossentropy to Predict Probability to actual class output, Stochastic Gradient Descent via Adam
cnn_model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy',
                           tf.keras.metrics.Precision(name='precision'),
                           tf.keras.metrics.Recall(name='recall')])

In [ ]:
# Summary of Model
cnn_model.summary()

In [ ]:
# Fit Model, 10 Epochs
cnn_history = cnn_model.fit(train_generator,
                        validation_data = valid_generator,
                        steps_per_epoch = train_generator.n//train_generator.batch_size,
                        validation_steps = valid_generator.n//valid_generator.batch_size,
                        epochs=10)

# Stop time of model
end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
# Save model locally
cnn_model.save("CNN_Initial_Model")

In [ ]:
# Visualize Model Metrics
visualize_training_results(cnn_history)

In [ ]:
# Evaluate against Training set
cnn_results_train = cnn_model.evaluate(train_generator)
cnn_results_train

In [ ]:
# Evaluate against Test set
cnn_results_test = cnn_model.evaluate(test_generator)
cnn_results_test

## Finetune Models

### MLP Finetune

In [ ]:
# Start time of model
original_start = datetime.datetime.now()
start = datetime.datetime.now()

In [ ]:
# Checkpoint, Monitor Loss, Save best results
mlp_checkpoint = tf.keras.callbacks.ModelCheckpoint("MLP_Finetune_Checkpoint",
                                                monitor="val_loss",
                                                save_best_only=True)

# Early stop after 10 efforts to learn, each Epoch starts with best saved weights
mlp_early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,
                                                  monitor="val_loss",
                                                  restore_best_weights=True)

In [ ]:
# Fit Model, 100 Epochs, Checkpoint and Early Stop
mlp_finetune_history = mlp_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    epochs=100,
    validation_data=valid_generator,
    validation_steps = valid_generator.n//valid_generator.batch_size,
    callbacks=[mlp_checkpoint, mlp_early_stopping]
)

# Stop time of model
end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
# Visualize Model Metrics
visualize_training_results(mlp_finetune_history)

In [ ]:
# Evaluate model against Training set
mlp_results_train = mlp_model.evaluate(train_generator)

In [ ]:
# Evaluate model again Test set
mlp_results_test = mlp_model.evaluate(test_generator)

### CNN Finetune

In [ ]:
# Start time of model
original_start = datetime.datetime.now()
start = datetime.datetime.now()

In [ ]:
# Checkpoint, Monitor Loss, Save best results
cnn_checkpoint = tf.keras.callbacks.ModelCheckpoint("CNN_Finetune_Checkpoint",
                                                monitor="loss",
                                                save_best_only=True)

# Early stop after 10 efforts to learn, each Epoch starts with best saved weights
cnn_early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,
                                                  monitor="loss",
                                                  restore_best_weights=True)

In [ ]:
# Fit Model, 100 Epochs, Checkpoint and Early Stop
cnn_finetune_history = cnn_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    epochs=100,
    validation_data=valid_generator,
    validation_steps = valid_generator.n//valid_generator.batch_size,
    callbacks=[cnn_checkpoint, cnn_early_stopping]
)

# Stop time of model
end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
# Visualize Model Metrics
visualize_training_results(cnn_finetune_history)

In [ ]:
# Evaluate against Training set
cnn_results_train = cnn_model.evaluate(train_generator)

In [ ]:
# Evaluate against Test set
cnn_results_test = cnn_model.evaluate(test_generator)

## Pretrained Model

In [ ]:
# Start time of model
original_start = datetime.datetime.now()
start = datetime.datetime.now()

In [ ]:
# Initialize Pretrain Base Model
pretrain_base = DenseNet201(weights=None,
                 include_top=False,
                 input_shape=(200, 200, 3))

# Define Model Architecture
pretrain_model = keras.Sequential()
pretrain_model.add(pretrain_base)
pretrain_model.add(keras.layers.Flatten())
pretrain_model.add(keras.layers.Dense(64, activation='relu'))
pretrain_model.add(keras.layers.Dense(128, activation='relu'))
pretrain_model.add(keras.layers.Dense(256, activation='relu'))
pretrain_model.add(keras.layers.Dense(128, activation='relu'))
pretrain_model.add(keras.layers.Dense(1, activation='sigmoid'))

# Do NOT train the base
pretrain_base.trainable = False

# Summary of Model
pretrain_model.summary()

In [ ]:
# Compile. Binary Crossentropy to Predict Probability to actual class output, Stochastic Gradient Descent via Adam
pretrain_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy',
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall')])

In [ ]:
# Fit Model, 10 Epochs
pretrain_model_history = pretrain_model.fit(train_generator,
                                            validation_data = valid_generator,
                                            steps_per_epoch = train_generator.n//train_generator.batch_size,
                                            validation_steps = valid_generator.n//valid_generator.batch_size,
                                            epochs=10)

# Stop time of model
end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
# Save model locally
pretrain_model.save("Pretrain_Initial_Model")

In [ ]:
# Visualize Model Metrics
visualize_training_results(pretrain_model_history)

In [ ]:
# Evaluate model against Training set
pretrain_results_train = pretrain_model.evaluate(train_generator)
pretrain_results_train

In [ ]:
# Evaluate model against Test set
pretrain_results_test = pretrain_model.evaluate(test_generator)
pretrain_results_test

## Finetune Pretrain

In [ ]:
# Start time of model
original_start = datetime.datetime.now()
start = datetime.datetime.now()

In [ ]:
# Checkpoint, Monitor Loss, Save best results
pretrain_model_checkpoint = tf.keras.callbacks.ModelCheckpoint("Finetune_Pretrain_Checkpoint",
                                                               monitor="loss",
                                                               save_best_only=True)

# Early stop after 10 efforts to learn, each Epoch starts with best saved weights
pretrain_model_early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,
                                                                 monitor="loss",
                                                                 restore_best_weights=True)

In [ ]:
# Initialize Pretrain Base Model
finetune_pretrain_base = DenseNet201(weights=None,
                                     include_top=False,
                                     input_shape=(200, 200, 3))

# Define Model Architecture
finetune_pretrain_base = keras.Sequential()
finetune_pretrain_base.add(pretrain_base)
finetune_pretrain_base.add(keras.layers.Flatten())
finetune_pretrain_base.add(keras.layers.Dense(64, activation='relu'))
finetune_pretrain_base.add(keras.layers.Dense(128, activation='relu'))
finetune_pretrain_base.add(keras.layers.Dense(256, activation='relu'))
finetune_pretrain_base.add(keras.layers.Dense(128, activation='relu'))
finetune_pretrain_base.add(keras.layers.Dense(1, activation='sigmoid'))

# Do NOT train the base
finetune_pretrain_base.trainable = False

# Summary of Model
finetune_pretrain_base.summary()

In [ ]:
# Compile. Binary Crossentropy to Predict Probability to actual class output, Stochastic Gradient Descent via Adam
finetune_pretrain_base.compile(loss="binary_crossentropy",
                               optimizer="adam",
                               metrics=['accuracy',
                                        tf.keras.metrics.Precision(name='precision'),
                                        tf.keras.metrics.Recall(name='recall')])

In [ ]:
# Fit Model, 100 Epochs, Checkpoint and Early Stop
finetune_pretrain_model_history = pretrain_model.fit(train_generator,
                                                     validation_data = valid_generator,
                                                     steps_per_epoch = train_generator.n//train_generator.batch_size,
                                                     validation_steps = valid_generator.n//valid_generator.batch_size,
                                                     callbacks=[pretrain_model_checkpoint, pretrain_model_early_stopping],
                                                     epochs=100)

# Stop time of model
end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
# Visualize Model Metrics
visualize_training_results(finetune_pretrain_model_history)

In [ ]:
# Evaluate Model against Training set
finetune_pretrain_results_train = pretrain_model.evaluate(train_generator)
finetune_pretrain_results_train

In [ ]:
# Evaluate Model against Test set
finetune_pretrain_results_test = pretrain_model.evaluate(test_generator)
finetune_pretrain_results_test

# EVALUATION

In [ ]:
# Load saved model
mlp_original_model = keras.models.load_model('MLP_Initial_Model')

In [ ]:
scores = mlp_original_model.evaluate(test_generator)
print("Loss of the model: %.2f%%"%(scores[0] * 100))
print("Test Accuracy: %.2f%%"%(scores[1] * 100))
print("Test Precision: %.2f%%"%(scores[2] * 100))
print("Test Recall: %.2f%%"%(scores[3] * 100))

In [ ]:
# Load saved model
mlp_finetune_model = keras.models.load_model('MLP_Finetune_Checkpoint')

In [ ]:
# Finalized, Organized, Evaluate against Test set with metrics
scores = mlp_finetune_model.evaluate(test_generator)
print("Loss of the model: %.2f%%"%(scores[0] * 100))
print("Test Accuracy: %.2f%%"%(scores[1] * 100))
print("Test Precision: %.2f%%"%(scores[2] * 100))
print("Test Recall: %.2f%%"%(scores[3] * 100))

In [ ]:
# Load saved model
cnn_initial_model = keras.models.load_model('CNN_Initial_Model')

In [ ]:
# Finalized, Organized, Evaluate against Test set with metrics
scores = cnn_initial_model.evaluate(test_generator)
print("Loss of the model: %.2f%%"%(scores[0] * 100))
print("Test Accuracy: %.2f%%"%(scores[1] * 100))
print("Test Precision: %.2f%%"%(scores[2] * 100))
print("Test Recall: %.2f%%"%(scores[3] * 100))

In [ ]:
# Load saved model
cnn_finetune_model = keras.models.load_model('CNN_Finetune_Checkpoint')

In [ ]:
# Finalized, Organized, Evaluate against Test set with metrics
scores = cnn_finetune_model.evaluate(test_generator)
print("Loss of the model: %.2f%%"%(scores[0] * 100))
print("Test Accuracy: %.2f%%"%(scores[1] * 100))
print("Test Precision: %.2f%%"%(scores[2] * 100))
print("Test Recall: %.2f%%"%(scores[3] * 100))

In [ ]:
# Load saved model
pretrain_model = keras.models.load_model('Pretrain_Initial_Model')

In [ ]:
# Finalized, Organized, Evaluate against Test set with metrics
scores = pretrain_model.evaluate(test_generator)
print("Loss of the model: %.2f%%"%(scores[0] * 100))
print("Test Accuracy: %.2f%%"%(scores[1] * 100))
print("Test Precision: %.2f%%"%(scores[2] * 100))
print("Test Recall: %.2f%%"%(scores[3] * 100))

In [ ]:
# Load saved model
pretrain_finetune_model = keras.models.load_model('Finetune_Pretrain_Checkpoint')

In [ ]:
# Finalized, Organized, Evaluate against Test set with metrics
scores = pretrain_finetune_model.evaluate(test_generator)
print("Loss of the model: %.2f%%"%(scores[0] * 100))
print("Test Accuracy: %.2f%%"%(scores[1] * 100))
print("Test Precision: %.2f%%"%(scores[2] * 100))
print("Test Recall: %.2f%%"%(scores[3] * 100))

In [ ]:
# Notebook Stop, Total Time.
notebook_end = datetime.datetime.now()
notebook_elapsed = notebook_end - notebook_original_start
print('Notebook took a total of {}'.format(elapsed))